# Quick-Start Guide

**MLRun** is an end-to-end [open-source](https://github.com/mlrun/mlrun) MLOps solution to manage and automate your
entire analytics and machine learning lifecycle, from data ingestion, through model development to full pipeline deployment.
MLRun is running as a built-in service in Iguazio and is integrated well with other services in the platform.
Its primary goal is to ease the development of machine learning pipeline at scale and help organizations build a
robust process for moving from the research phase to fully operational production deployments.

**Table of Contents**

- [Working with MLRun](#working-with-mlrun)
  - [Train a Model](#train-a-model)
  - [Test the Model](#test-the-model)
  - [Serve the Model](#serve-the-model)

## Working with MLRun
<a name="working-with-mlrun"></a>

If you need to install MLRun, refer to the [Installation Guide](install.md).
>**Note**: If you are using the [Iguazio Data Science Platform](https://www.iguazio.com/), MLRun already comes
>preinstalled and integrated in your system.

If you are not viewing this quick-start guide from a Jupyter Lab instance, open it on your cluster, create a
new notebook, and copy the sections below to the notebook to run them.

### Set Environment

Before you begin, initialize MLRun by calling `set_environment` and provide it the project name

In [1]:
from mlrun import set_environment

project_name = 'quick-start'
_, artifact_path = set_environment(project=project_name)

## Train a Model
<a name="train-a-model"></a>

MLRun introduces the concept of [functions](./runtimes/functions.md). You can run your own code as functions, or use
functions from the function marketplace.
In the example below, we'll use the [`sklearn_classifier`](https://github.com/mlrun/functions/tree/master/sklearn_classifier)
from the function marketplace to train a model.

In [2]:
from mlrun import import_function
from mlrun.platforms import auto_mount

train = import_function('hub://sklearn_classifier').apply(auto_mount())

train_run = train.run(name='train',
                      inputs={'dataset': 'https://s3.wasabisys.com/iguazio/data/iris/iris_dataset.csv'},
                      params={'model_pkg_class': 'sklearn.linear_model.LogisticRegression',
                              'label_column': 'label'},
                      project=project_name)

The run output above contains a link to the MLRun UI. Click it to inspect the various aspects of the jobs you run:

<img src="./_static/images/mlrun-quick-start/train-info.png" alt="ui-info" width="800"/>

As well as their artifacts:

<img src="./_static/images/mlrun-quick-start/train-artifacts.png" alt="ui-artifacts" width="800"/>

When running the function in a Jupyter notebook, the output cell for your function execution will contain a table with
run information &mdash; including the state of the execution, all inputs and parameters, and the execution results and artifacts.

![MLRun quick start train output](./_static/images/mlrun-quick-start/train-output.png)

## Test the Model
<a name="test-the-model"></a>

Now that you have a trained model, you can test it: run a task that uses the [test_classifier](https://github.com/mlrun/functions/tree/master/test_classifier)
function from the function marketplace to run the selected trained model against the test dataset. The test dataset
was returned from the training task (`train_run`) in the previous step.

In [3]:
test = import_function('hub://test_classifier').apply(auto_mount())

You can then run the function as part of your project, just as any other function that you have written yourself.
To view the function documentation, call the `doc` method:

In [4]:
test.doc()

function: test-classifier
test a classifier using held-out or new data
default handler: test_classifier
entry points:
  test_classifier: Test one or more classifier models against held-out dataset

Using held-out test features, evaluates the peformance of the estimated model

Can be part of a kubeflow pipeline as a test step that is run post EDA and 
training/validation cycles
    context  - the function context, default=
    models_path(DataItem)  - artifact models representing a file or a folder, default=
    test_set(DataItem)  - test features and labels, default=
    label_column(str)  - column name for ground truth labels, default=
    score_method(str)  - for multiclass classification, default=micro
    plots_dest(str)  - dir for test plots, default=
    model_evaluator  - NOT IMPLEMENTED: specific method to generate eval, passed in as string or available in this folder, default=None
    default_model(str)  - , default=model.pkl
    predictions_column(str)  - column name for the 

Configure parameters for the test function (`params`), and provide the selected trained model from the train task as an input artifact (`inputs`)

In [5]:
test_run = test.run(name="test",
                    params={"label_column": "label"},
                    inputs={"models_path": train_run.outputs['model'],
                            "test_set": train_run.outputs['test_set']},
                    project=project_name)

## Serve the Model
<a name="serve-the-model"></a>

MLRun serving can take MLRun models or standard model files and produce managed, real-time, serverless functions using
the [Nuclio real-time serverless framework](https://www.iguazio.com/open-source/nuclio/).
Nuclio is built around data, I/O, and compute intensive workloads and is focused on performance and flexibility.
Nuclio is also deeply integrated into the MLRun framework.
See [MLRun Serving documentation](./serving/serving-graph.md) to learn more about the rich serving capabilities
MLRun has to offer.


To deploy your model using the [v2_model_server function](https://github.com/mlrun/functions/tree/master/v2_model_server),
run the following code:

In [6]:
serve = import_function('hub://v2_model_server').apply(auto_mount())
model_name='iris'
serve.add_model(model_name, model_path=train_run.outputs['model'])
addr = serve.deploy(project=project_name)

The `invoke` method enables to programmatically test the function.

In [7]:
import json

inputs = [[5.1, 3.5, 1.4, 0.2],
          [7.7, 3.8, 6.7, 2.2]]
my_data = json.dumps({'inputs': inputs})
serve.invoke(f'v2/models/{model_name}/infer', my_data)

Open the Nuclio UI to view the function and test it.

![Nuclio Functions UI](./_static/images/mlrun-quick-start/serve.png)

For a more detailed walk-through, refer to the [getting-started tutorial](tutorial/index.md).